<img src='https://upload.wikimedia.org/wikipedia/fr/thumb/e/ed/Logo_Universit%C3%A9_du_Maine.svg/1280px-Logo_Universit%C3%A9_du_Maine.svg.png' width="300" height="500">
<br>
<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Infrastructures pour l'IA</h1>
    <p style='text-align: center;'>Master Informatique 1</p>
    <p style='text-align: center;'>Anhony Larcher</p>
</div>

Cette session est organisée comme un challenge:
* Vous optimiserez un code afin d'effectuer un calcul le plus rapidement possible
* le résultat sera utilisée dans le TP numéro 6 pour faire du calcul parallélisé.


# Implémenter le calcul suivant et optimisez le au mieux

Il s'agit d'accumuler les statistiques d'ordre 0 et d'ordre 1 sur un mélange de Gaussiennes. Vous trouverez sur Umtice une archive zip contenant un objet Mixture et des paramètres acoustiques de type MFCC.

Chaque distribution de la mixture a pour densité de probabilité:

Et la densité de probabilité du mélange de Gaussienne est 
$$p(x|\lambda) = \Sigma_{i=1}^{M}w_ip_i(x)$$

où $$\Sigma_{i=1}^{M}w_i = 1$$

et 

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i))}$$

Nous souhaiton ici calculer pour chaque trame $x_t$

$$P(i|x_t) = \frac{w_ip_i(x_t)}{\Sigma_{j=1}^M w_j p_j(x(t))}$$

Pour ensuite calculer les statistiques d'ordre 0 pour chaque gaussienne $i$:

$$n_i = \Sigma_{t=1}^{T}P(i|x_t)$$

Les statistiques d'ordre 1:

$$E_i(x) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t$$

Et les statistiques d'ordre 2:

$$E_i(x^2) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t^2$$

## 1.1) Initialisez l'accumulateur de statistiques

Le fichier ``mixture.py``contient le code d'une classe Mixtuyre qui sera utilisé dans ce TP. Ce code se trouve également dans la cellule ci-dessous pour travailler dans le notebook. Une fois la cellule ci-dessous complétée vous pourrez recopier le code dans le fichier ``mixture.py``afin d'avoir une version propre du code.

In [37]:
import numpy
import pickle


class Mixture(object):

    def __init__(self):
        """
        Initialize an empty Mixture object
        """
        self.w = numpy.array([])
        self.mu = numpy.array([])
        self.invcov = numpy.array([])
        self.cst = numpy.array([])
        self.det = numpy.array([])

    def __repr__(self):
        """
        Serialize a Mixture object to text
        """
        return f'w = {self.w}\nmu =  {selmf.mu}\ninvcov = {self.invcov}, cst = {self.cst}\ndet = {self.det}'

    @classmethod
    def read(cls, filename):
        """
        Read a Mixture object stored in Pickle format on disk
        :param filename: the name of the file to read from
        :return: a Mixture object
        """
        with open(filename, 'rb') as fh:
            mixture = pickle.load(fh)
            return mixture

    def save(self, filename):
        """
        Save a Mixture object to disk in Pickle format
        :param filename: the name of the file to write to
        """
        with open(filename, 'wb') as fh:
            pickle.dump(self, fh)

    def compute_lpp_1(self, cep, mu=None):
        """ Compute log posterior probabilities for a set of feature frames.

        :param cep: a set of feature frames in a ndarray, one feature per row
        :param mu: a mean super-vector to replace the ubm's one. If it is an empty 
              vector, use the UBM, dimension is: 

        :return: A ndarray of log-posterior probabilities corresponding to the 
              input feature set.
        """
        mu = self.mu
        # Get the dimension of the feature vector from the cep input
        self.dim = self.mu.shape[1]

        # Compute the value of the determinant from the covariance matrix
        self.det = numpy.zeros(64)
        for ii in range(self.dim):
            self.det[ii] = 1.0 / numpy.prod(self.invcov[ii, :])

        # for each Guassian distribution, compute the term that is invariant
        self.cst = numpy.zeros(64)
        for ii in range(self.dim):
            self.cst[ii] = 1.0 / (numpy.sqrt(self.det[ii]) * (2.0 * numpy.pi) ** (self.dim / 2.0))

        A = numpy.zeros(64)
        # For each feature, compute the data independent term
        for ii in range(self.dim):
            A[ii] = (numpy.square(self.mu[ii, :]) * self.invcov[ii, :]).sum() - 2.0 * (numpy.log(self.w[ii]) + numpy.log(self.cst[ii]))

        # Compute the data dependent term
        # numpy.square(cep) est de dimension Nx39
        # numpy.dot(numpy.square(cep), self.invcov.T) est de dimension Nx39 multiplié par 39x64 -> dimension Nx64
        # mu.reshape(self.mu.shape) * self.invcov) est de dimension 64x39 
        # numpy.transpose(mu.reshape(self.mu.shape) * self.invcov) est de dimension 39x64
        # numpy.dot(cep, numpy.transpose(mu.reshape(self.mu.shape) * self.invcov)) est de dimension Nx39 multiplié par 39x64 -> Nx64
        # B est de dimension Nx64

        B = numpy.zeros((cep.shape[0], 64))
        # for each feature and each distribution
        for tt in range(cep.shape[0]):
            for ii in range(self.dim):
                B[tt, ii] = numpy.dot(numpy.square(cep[tt, :]), self.invcov[ii, :].T) - 2.0 * numpy.dot(cep[tt, :], numpy.transpose(mu[ii, :].reshape(self.dim) * self.invcov[ii, :]))


        # Compute the exponential term
        lp = -0.5 * (B + A)
        #for tt in range(cep.shape[0]):
        #    for ii in range(self.dim):
        #        pass
        return lp

    def compute_log_posterior_probabilities(self, cep, mu=None):
        """ Compute log posterior probabilities for a set of feature frames.

        :param cep: a set of feature frames in a ndarray, one feature per row
        :param mu: a mean super-vector to replace the ubm's one. If it is an empty 
              vector, use the UBM, dimension is: 

        :return: A ndarray of log-posterior probabilities corresponding to the 
              input feature set.
        """

        self.dim = self.mu.shape[1]
        self.det = 1.0 / numpy.prod(self.invcov, axis=1)      # self.det est de dimension 64
        self.cst = 1.0 / (numpy.sqrt(self.det) * (2.0 * numpy.pi) ** (self.dim / 2.0))    # cst est de dimension 64

        mu = self.mu
        # for MAP, Compute the data independent term
        # self.mu est de dimension 64x39
        # self.invcov est de dimension 64x39
        # numpy.square(self.mu) * self.invcov) est de dimension 64x39
        # numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
        # A est de dimension 64
        A = (numpy.square(self.mu) * self.invcov).sum(1) - 2.0 * (numpy.log(self.w) + numpy.log(self.cst))


        # Compute the data dependent term
        #  numpy.square(cep) est de dimension Nx39
        # numpy.dot(numpy.square(cep), self.invcov.T) est de dimension Nx39 multiplié par 39x64 -> dimension Nx64
        # mu.reshape(self.mu.shape) * self.invcov) est de dimension 64x39 
        # numpy.transpose(mu.reshape(self.mu.shape) * self.invcov) est de dimension 39x64
        # numpy.dot(cep, numpy.transpose(mu.reshape(self.mu.shape) * self.invcov)) est de dimension Nx39 multiplié par 39x64 -> Nx64
        # B est de dimension Nx64
        B = numpy.dot(numpy.square(cep), self.invcov.T) - 2.0 * numpy.dot(cep, numpy.transpose(mu.reshape(self.mu.shape) * self.invcov))

        # Compute the exponential term
        lp = -0.5 * (B + A)
        return lp



In [2]:
import numpy
data = numpy.load("data_1/features_1.npy")
data.shape

(338, 39)

## Analyse des formules

On remarque d'abord que certains éléments dépendent des données, et plus particulièrement de chaque vecteur de donnée, mais également de chaque distribution de la mixture.

Nous verrons donc apparaitre 2 boucles principales:
* une sur les données
* une sur les distributions

Pour simplifier le calcul nous devons donc séparer ce qui est indépendant de ces boucles afin de ne pas le recalculer plusieurs fois.

Ré-écrivez $$p_i(x)$$ en séparant ces termes.

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i))}$$

On trouve d'abord un terme qui dépend de chaque mixture: leur déterminant

Nous pouvons maintenant calculer le premier terme pour chaque distribution:
    
$$ \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} $$ 

In [3]:
# Pour chaque distribution, le terme invariant est:

for ii in range(self.dim):
    self.cst[ii] = 1.0 / (numpy.sqrt(self.det[ii]) * (2.0 * numpy.pi) ** (self.dim / 2.0))

NameError: name 'self' is not defined

Regardons maintenant le terme contenu dans l'exponentielle:

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) $$ 

Tout ce terme dépend des distributions, une partie seulement dépend des données.

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) = -\frac{1}{2} ( x^T\Sigma_i^{-1}x + \mu_i^T\Sigma_i^{-1}\mu_i - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x ) $$ 

Calculez chacun de ces termes:

Analyse du terme independant des données:

$$ \mu_i^T\Sigma_i^{-1}\mu_i $$

$$ \mu_i^T$$ est de dimension (1, 39) $$\Sigma_i$$ est de dimension (39, 39)

Donc $$ \mu_i^T\Sigma_i^{-1}\mu_i $$ est de dimension (1,)

In [ ]:
# En pratique, comme la matruice de covariance est diagonale, nous n'avons pas besoin de calculer un vrai produit matriciel
# Écrivez la valeur du premier terme de \mu_i \Sigma^{-1}_i et tirez partie de cette expression pour simplifier
# le calcul de ce terme

# self.mu est de dimension 64x39
# self.invcov est de dimension 64x39
# numpy.square(self.mu) * self.invcov) est de dimension 64x39
# numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
# A est de dimension 64

for ii in range(self.dim):
    A_idpt[ii] = (numpy.square(self.mu[ii, :]) * self.invcov[ii, :]).sum(1)

# Allez plus loin en vectorisant entre toutes les distributions
A_idpt = (numpy.square(self.mu) * self.invcov).sum(1)

De la même façon calculez les termes dépendant des données

In [4]:
B = numpy.dot(numpy.square(cep), self.invcov.T) 
    - 2.0 * numpy.dot(cep, numpy.transpose(self.mu.reshape(self.mu.shape) * self.invcov))


IndentationError: unexpected indent (<ipython-input-4-8307b2353daf>, line 2)

In [5]:
ubm = Mixture()
ubm.w = numpy.load('w.npy')
ubm.mu = numpy.load('mu.npy')
ubm.invcov = numpy.load('invcov.npy')

# load data
data = numpy.load('features_0.npy')
data.shape

lp = ubm.compute_lpp_1(data)
lp2 = ubm.compute_log_posterior_probabilities(data)

FileNotFoundError: [Errno 2] No such file or directory: 'features_0.npy'

## 1.2) Écrivez une fonction compute_lpp 

Cette méthode de la class **Mixture** va calculer les log posterior probabilités $$\log{p_i(x)}$$ d'un ensemble de vecteurs sur ce mélange de Gaussienne à matrices de covariances diagonales

Pour acumuler les statistiques, vous allez créer un objet mixture que vous appelerez **accum**.

* Cette méthode prend en paramètres une matrice de coefficients cepstraux de dimension N x F où N est le nombre de vecteurs (variable selon les fichiers) et F est la dimension des vecteurs (39 dans notre cas)


### Première version

Dans un premier temps écrivez une version immédiate des équations ci-dessus afin d'avoir un premier code, facile à lire et qui permettra par la suite de vérifier l'exactitude du résultat et de comparer l'eficacité des différentes versions du code. 

In [6]:
def compute_lpp_1(self, cep, mu=None):
    """ Compute log posterior probabilities for a set of feature frames.

    :param cep: a set of feature frames in a ndarray, one feature per row
    :param mu: a mean super-vector to replace the ubm's one. If it is an empty 
          vector, use the UBM, dimension is: 

    :return: A ndarray of log-posterior probabilities corresponding to the 
          input feature set.
    """
    # Get the dimension of the feature vector from the cep input
    self.dim = self.mu.shape[1]
    
    # Compute the value of the determinant from the covariance matrix
    for ii in range(self.dim):
        self.det[ii] = 1.0 / numpy.prod(self.invcov[ii, :])
    
    # for each Guassian distribution, compute the term that is invariant
    for ii in range(self.dim):
        self.cst[ii] = 1.0 / (numpy.sqrt(self.det[ii]) * (2.0 * numpy.pi) ** (self.dim / 2.0))
    
    # For each feature, compute the data independent term
    for ii in range(self.dim):
        A[ii] = (numpy.square(self.mu[ii, :]) * self.invcov[ii, :]).sum() - 2.0 * (numpy.log(self.w[ii]) + numpy.log(self.cst[ii]))
    
    # Compute the data dependent term
    # numpy.square(cep) est de dimension Nx39
    # numpy.dot(numpy.square(cep), self.invcov.T) est de dimension Nx39 multiplié par 39x64 -> dimension Nx64
    # mu.reshape(self.mu.shape) * self.invcov) est de dimension 64x39 
    # numpy.transpose(mu.reshape(self.mu.shape) * self.invcov) est de dimension 39x64
    # numpy.dot(cep, numpy.transpose(mu.reshape(self.mu.shape) * self.invcov)) est de dimension Nx39 multiplié par 39x64 -> Nx64
    # B est de dimension Nx64
    
    # for each feature and each distribution
    for tt in range(cep.shape[0]):
        for ii in range(self.dim):
            B[tt, ii] = numpy.dot(numpy.square(cep[tt, :]), self.invcov[ii, :].T) - 2.0 * numpy.dot(cep[tt, :], numpy.transpose(mu[tt, :].reshape(self.dim) * self.invcov[ii, :]))


    # Compute the exponential term
    #lp = -0.5 * (B + A)
    for tt in range(cep.shape[0]):
        for ii in range(self.dim):
            pass
    return lp

* En premier lieu, pensez à extraire des boucles tous les termes qui ne dépendent pas des données
* Cette méthode renvoie les $$n_i$$ (donc un vecteur)

## Correction

* En premier lieu, cette méthode calcule le terme de l'équation qui est indépendant des données.

$$\Sigma(\mu^2*\Sigma^{-1}) - 2.*\log{w} + \log{cst}$$

Le calcul de $$\mu^T\Sigma^{-1}\mu$$ peut déjà être optimisé car la covariance est diagonale


In [7]:
def compute_log_posterior_probabilities(self, cep, mu=None):
    """ Compute log posterior probabilities for a set of feature frames.

    :param cep: a set of feature frames in a ndarray, one feature per row
    :param mu: a mean super-vector to replace the ubm's one. If it is an empty 
          vector, use the UBM, dimension is: 

    :return: A ndarray of log-posterior probabilities corresponding to the 
          input feature set.
    """

    self.dim = self.mu.shape[1]
    self.det = 1.0 / numpy.prod(self.invcov, axis=1)      # self.det est de dimension 64
    self.cst = 1.0 / (numpy.sqrt(self.det) * (2.0 * numpy.pi) ** (self.dim / 2.0))    # cst est de dimension 64
    
    # for MAP, Compute the data independent term
    # self.mu est de dimension 64x39
    # self.invcov est de dimension 64x39
    # numpy.square(self.mu) * self.invcov) est de dimension 64x39
    # numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
    # A est de dimension 64
    A = (numpy.square(self.mu) * self.invcov).sum(1) - 2.0 * (numpy.log(self.w) + numpy.log(self.cst))


    # Compute the data dependent term
    #  numpy.square(cep) est de dimension Nx39
    # numpy.dot(numpy.square(cep), self.invcov.T) est de dimension Nx39 multiplié par 39x64 -> dimension Nx64
    # mu.reshape(self.mu.shape) * self.invcov) est de dimension 64x39 
    # numpy.transpose(mu.reshape(self.mu.shape) * self.invcov) est de dimension 39x64
    # numpy.dot(cep, numpy.transpose(mu.reshape(self.mu.shape) * self.invcov)) est de dimension Nx39 multiplié par 39x64 -> Nx64
    # B est de dimension Nx64
    B = numpy.dot(numpy.square(cep), self.invcov.T) - 2.0 * numpy.dot(cep, numpy.transpose(mu.reshape(self.mu.shape) * self.invcov))

    # Compute the exponential term
    lp = -0.5 * (B + A)
    return lp



## 1.3) Utilisez la fonction **sum_lp**
* Que fait cette fonction qui vous est donnée?
* Pourquoi l'utilise-t'on?

In [8]:
def sum_log_probabilities(lp):
    """Sum log probabilities in a secure manner to avoid extreme values

    :param lp: numpy array of log-probabilities to sum
    """
    pp_max = numpy.max(lp, axis=1)
    log_lk = pp_max + numpy.log(numpy.sum(numpy.exp((lp.transpose() - pp_max).T), axis=1))
    ind = ~numpy.isfinite(pp_max)
    if sum(ind) != 0:
        log_lk[ind] = pp_max[ind]
    pp = numpy.exp((lp.transpose() - log_lk).transpose())
    llk = log_lk.sum()
    return pp, llk

# Mesure du temps de calcul

## 1.4) Bouclez sur les fichiers de paramètres pour accumuler les statistiques

In [39]:
ubm = Mixture()
ubm.w = numpy.load('w.npy')
ubm.mu = numpy.load('mu.npy')
ubm.invcov = numpy.load('invcov.npy')

accum = Mixture()
accum.w = numpy.zeros(ubm.w.shape)
accum.cst = numpy.zeros(ubm.w.shape)
accum.det = numpy.zeros(ubm.w.shape)
accum.w = numpy.zeros(ubm.w.shape)
accum.mu = numpy.zeros(ubm.mu.shape)
accum.invcov = numpy.zeros(ubm.invcov.shape)

#for ii in range(100):
ii = 0
data = numpy.load(f'data_1/features_{ii}.npy')

import time
start_time = time.time()
lp = ubm.compute_lpp_1(data)
end_time = time.time()
elapsed_time_1 = end_time - start_time
print("Elapsed time 1: ", elapsed_time_1) 


start_time = time.time()
lp = ubm.compute_log_posterior_probabilities(data)
end_time = time.time()
elapsed_time_2 = end_time - start_time
print("Elapsed time 2: ", elapsed_time_2) 

print(f'Gain = {elapsed_time_1 / elapsed_time_2}')

Elapsed time 1:  1.457082986831665
Elapsed time 2:  0.0023429393768310547
Gain = 621.9038363691869


In [ ]:
# Correction
# zero order statistics
accum.w += pp.sum(0)
# first order statistics
accum.mu += numpy.dot(cep.T, pp).T
# second order statistics
accum.invcov += numpy.dot(numpy.square(cep.T), pp).T  # version for diagonal covariance


In [28]:
ubm.invcov = numpy.load('invcov.npy')
ubm.invcov

array([[0.57644626, 0.68576746, 0.62941113, ..., 0.75738399, 0.57559068,
        0.66871915],
       [0.80763037, 0.76329883, 0.69521484, ..., 0.87043643, 0.69031538,
        0.74854336],
       [0.81601115, 0.76350196, 0.72462563, ..., 0.81519951, 0.81604256,
        0.86340996],
       ...,
       [0.72634523, 0.59532771, 0.71438645, ..., 0.72438607, 0.7639669 ,
        0.77606345],
       [0.71545848, 0.84455897, 0.84399906, ..., 0.63912259, 0.78547948,
        0.91511047],
       [0.59150909, 0.61340457, 0.68649622, ..., 0.61529548, 0.71058816,
        0.57820938]])